In [3]:
from transformers import BertModel, AutoConfig, AutoTokenizer
import torch.nn as nn
import datasets
import csv

In [11]:

model_name = "bert-base-uncased"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)


In [12]:

# Add linear layer
output_size = 1  
linear_layer = nn.Linear(config.hidden_size, output_size)  

# Combine BERT and the linear layer
class BertWithLinear(nn.Module):
    def __init__(self):
        super(BertWithLinear, self).__init__()
        self.bert = bert_model
        self.linear = linear_layer

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # Use pooled output for classification/regression
        pooled_output = output[1] 
        return self.linear(pooled_output) 

model = BertWithLinear()

In [13]:
# freeze BERT pretrained weights
for param in model.bert.embeddings.parameters():
    param.requires_grad = False

In [24]:
# Load dataset
feats_fp = open("BERT_X.csv", "r")
labels_fp = open("BERT_y.csv", "r")
feats = csv.reader(feats_fp)
labels = csv.reader(labels_fp)

# skip header
next(feats)
next(labels)

data = {'text': [], 'label': []}
for row in feats:
    data['text'].append(row[0].strip().replace("\n", " "))
for row in labels:
    data['label'].append(float(row[0].strip().replace("\n", "")))
print(len(data['text']), len(data['label']))
assert len(data['text']) == len(data['label'])
dataset = datasets.Dataset.from_dict(data)


AttributeError: 'list' object has no attribute 'strip'

In [18]:
print(dataset[0])

{'text': ['Overview\n\nHearingLife is a national hearing care company and part of the Demant Group, a global leader in hearing healthcare built on a heritage of care, health, and innovation since 1904. HearingLife operates more than 600 hearing care centers across 42 states. We follow a scientific, results-oriented approach to hearing healthcare that is provided by highly skilled and caring professionals. Our vision is to help more people hear better through life-changing hearing health delivered by the best personalized care. This Team Member must uphold the HearingLife Core Values:\n\n We create trust  We are team players  We apply a can-do attitude  We create innovative solutions \n\nResponsibilities\n\nYou will help more people hear better by providing clinical expertise to diagnose and treat hearing loss while ensuring a positive patient experience. The Hearing Care Provider acts in accordance with required industry and state professional licensing standards and local practice sco

In [15]:
# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/27513 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]